# Suo

In [1]:
import warnings
import os
import sys
import gc
import warnings

In [2]:
import anndata as ad
import scanpy as sc
import copy
import torch
from pathlib import Path
import networkx as nx
from sklearn.neighbors import kneighbors_graph
import numpy as np
import scanpy as sc
import pandas as pd
import numpy as np
import scipy.stats
from scib_metrics.benchmark import Benchmarker, BioConservation, BatchCorrection
import scvi

sys.path.append("/home/icb/kemal.inecik/work/codes/tardis")
import tardis
tardis.config = tardis.config_server

sc.settings.verbosity = 3

In [3]:
torch.cuda.is_available()

False

In [4]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
import seaborn as sns

import pickle
_rcparams_path = "/home/icb/kemal.inecik/work/codes/tardis/training/local/figures/rcparams.pickle"
with open(_rcparams_path, 'rb') as file:
    _rcparams = pickle.load(file)
plt.rcParams.update(_rcparams)

In [5]:
adata = ad.read_h5ad("/home/icb/kemal.inecik/lustre_workspace/tardis_data/processed/dataset_complete_Suo_invae.h5ad")
obsms = [i for i in adata.obsm if i not in ['Unintegrated', 'X_pca', 'harmony']]
adata

AnnData object with n_obs × n_vars = 841922 × 8192
    obs: 'sample_ID', 'organ', 'age', 'cell_type', 'sex', 'sex_inferred', 'concatenated_integration_covariates', 'integration_donor', 'integration_biological_unit', 'integration_sample_status', 'integration_library_platform_coarse', 'n_genes'
    uns: 'rank_genes_groups'
    obsm: 'Unintegrated', 'X_pca', 'harmony', 'inveriant_1', 'inveriant_2', 'inveriant_3', 'inveriant_4', 'inveriant_5', 'inveriant_6', 'inveriant_7', 'inveriant_8'

In [6]:
biocons = BioConservation()
batchcor = BatchCorrection(
    silhouette_batch=False,
    ilisi_knn=False,
    kbet_per_label=False,
    graph_connectivity=False,
    pcr_comparison=False
)

In [7]:
def get_results(self, min_max_scale: bool = True) -> pd.DataFrame:
    _LABELS = "labels"
    _BATCH = "batch"
    _X_PRE = "X_pre"
    _METRIC_TYPE = "Metric Type"
    _AGGREGATE_SCORE = "Aggregate score"
    
    df = self._results.transpose()
    df.index.name = "Embedding"
    df = df.loc[df.index != _METRIC_TYPE]
    if min_max_scale:
        # Use sklearn to min max scale
        df = pd.DataFrame(
            MinMaxScaler().fit_transform(df),
            columns=df.columns,
            index=df.index,
        )
    df = df.transpose()
    df[_METRIC_TYPE] = self._results[_METRIC_TYPE].values

    # Compute scores
    per_class_score = df.groupby(_METRIC_TYPE).mean().transpose()
    # This is the default scIB weighting from the manuscript
    # per_class_score["Total"] = 0.4 * per_class_score["Batch correction"] + 0.6 * per_class_score["Bio conservation"]
    df = pd.concat([df.transpose(), per_class_score], axis=1)
    df.loc[_METRIC_TYPE, per_class_score.columns] = _AGGREGATE_SCORE
    return df

In [8]:
print(adata, flush=True)
print("starting scvi scib", flush=True)
bm = Benchmarker(
    adata=adata,
    batch_key="concatenated_integration_covariates",
    label_key="cell_type",
    embedding_obsm_keys=obsms,
    pre_integrated_embedding_obsm_key="Unintegrated",  # equals to X_pca
    bio_conservation_metrics=biocons,
    batch_correction_metrics=batchcor,
    n_jobs=-1,
)

AnnData object with n_obs × n_vars = 841922 × 8192
    obs: 'sample_ID', 'organ', 'age', 'cell_type', 'sex', 'sex_inferred', 'concatenated_integration_covariates', 'integration_donor', 'integration_biological_unit', 'integration_sample_status', 'integration_library_platform_coarse', 'n_genes'
    uns: 'rank_genes_groups'
    obsm: 'Unintegrated', 'X_pca', 'harmony', 'inveriant_1', 'inveriant_2', 'inveriant_3', 'inveriant_4', 'inveriant_5', 'inveriant_6', 'inveriant_7', 'inveriant_8'


starting scvi scib


In [9]:
bm.benchmark()

Computing neighbors:   0%|          | 0/8 [00:00<?, ?it/s]

Computing neighbors:  12%|█▎        | 1/8 [05:01<35:13, 301.91s/it]

Computing neighbors:  25%|██▌       | 2/8 [10:13<30:47, 307.88s/it]

Computing neighbors:  38%|███▊      | 3/8 [15:08<25:08, 301.67s/it]

Computing neighbors:  50%|█████     | 4/8 [20:16<20:17, 304.31s/it]

...

Metrics:  30%|███       | 3/10 [52:47<2:04:20, 1065.83s/it, Bio conservation: silhouette_label]

Metrics:  30%|███       | 3/10 [52:47<2:04:20, 1065.83s/it, Bio conservation: clisi_knn]       

Metrics:  40%|████      | 4/10 [52:51<1:04:39, 646.52s/it, Bio conservation: clisi_knn] 

Embeddings: 100%|██████████| 8/8 [6:53:58<00:00, 3147.62s/it]

Embeddings: 100%|██████████| 8/8 [6:53:58<00:00, 3104.82s/it]

In [10]:
df = get_results(bm, min_max_scale=False)

In [11]:
df

,isolated_labels,nmi_ari_cluster_labels_kmeans_nmi,nmi_ari_cluster_labels_kmeans_ari,silhouette_label,clisi_knn,Bio conservation
Embedding,,,,,,
inveriant_1,0.77074,0.611843,0.176302,0.502593,0.998185,0.611933
inveriant_2,0.860148,0.684053,0.218373,0.523225,0.998811,0.656922
inveriant_3,0.797439,0.617555,0.17428,0.494741,0.998307,0.616464
inveriant_4,0.847575,0.685539,0.212955,0.516162,0.99875,0.652196
inveriant_5,0.824615,0.644701,0.194351,0.495214,0.998217,0.631419
inveriant_6,0.829747,0.702215,0.239431,0.517052,0.9987,0.657429
inveriant_7,0.798021,0.650692,0.190949,0.508063,0.998821,0.629309
inveriant_8,0.838387,0.716064,0.251012,0.530873,0.99918,0.667103
Metric Type,Bio conservation,Bio conservation,Bio conservation,Bio conservation,Bio conservation,Aggregate score


In [12]:
pickle_path = os.path.join(tardis.config.io_directories["temp"], "latent", "suo_scib_invae_bio.pickle")
pickle_path

'/lustre/groups/ml01/workspace/kemal.inecik/tardis_data/_temporary/latent/suo_scib_invae_bio.pickle'

In [13]:
df.to_pickle(pickle_path)